# Using SQL with Python

As always, our first step is to make the necessary imports.  

In [1]:
# Standard data analysis packages
import numpy as np
import pandas as pd

# Packages for working with SQL Server
import sqlalchemy as db
import pyodbc

Now that we have that out of the way, we can use SQLAlchemy to create an **engine** object.  An engine is an object that represents a relational database and is the primary object we interact with to work with a database.  We create an engine using the following syntax:

```python
engine = db.create_engine(connection_str)
```

The connection string is a URL the engine uses to gain access to the database you're connecting to.  It can take on a number of forms, depending on which method you're using to establish a connection.  For the sake of simplicity (and/or sanity), I'll simply include the general forms of the connection strings and note that some Googling may be required to configure your connection string.

```python
# Hostname connection
conn_str = f"{db_type}+{db_connector}://{user_id}:{password}@{host}:{port}/{db_name}"

# DSN connection
conn_str = f"{db_type}+{db_connector}://{user_id}:{password}@{dsn}"
```


| Dialect | db_type Value |
| -- | -- |
| MySQL |  `mysql` | 
| PostgreSQL | `postgresql` |
| SQLite | `sqlite` |
| Oracle | `oracle` |
| Microsoft SQL | `mssql` |

| Dialect | db_connector Library |
| -- | -- |
| MySQL |  `pymysql`, `mysqldb` | 
| PostgreSQL | `psycopg2`, `pg8000` |
| SQLite | None needed |
| Oracle | `cx_oracle` |
| Microsoft SQL | `pymssql`, `pyodbc` |

When using either a hostname connection or a DSN connection, you simply pass conn_str to create_engine().  I'll be connecting to an instance of SQL Server on my local machine using Windows authentification, so my connection string will be a little different.

In [2]:
# Variables for connection string parameters
db_type = "mssql"
db_connector = "pyodbc"
server_name = "MATT-PC"
db_name = "AP"

# Creating connection string
conn_str = f"{db_type}+{db_connector}://{server_name}/{db_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"

# Create engine and connection objects
engine = db.create_engine(conn_str)
conn = engine.connect()

After you've successfully created for an engine, you may want to create a simple class to streamline the processs.  The class should be stored in a separate script so you can simply import it as needed.  You can password protect it to add a safeguard...

In [3]:
engine.table_names()

<ipython-input-3-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['ContactUpdates',
 'GLAccounts',
 'InvoiceArchive',
 'InvoiceLineItems',
 'Invoices',
 'sysdiagrams',
 'Terms',
 'Vendors']

In [4]:
import time

class sql_connector():
    def __init__(self):    
        self.stored_password = "my_password"
        self.input_password = input("Please enter password:\n\t")

    def create_connector(self):
        dots = "..."
        print("Validating password", end="")
        for dot in dots:
            time.sleep(1)
            print(dot, end="")
            
        time.sleep(1)
            
        if self.input_password == self.stored_password:
            print("\n\nConnection established")
            conn_str = f"{db_type}+{db_connector}://{server_name}/{db_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
            
            engine = db.create_engine(conn_str)
            conn = engine.connect()
            
            return engine, conn
        else:
            raise Exception("Incorrect password.")    

In [5]:
connector = sql_connector()
engine, conn = connector.create_connector()

Validating password...

Exception: Incorrect password.

In [6]:
connector = sql_connector()
engine, conn = connector.create_connector()

Validating password...

Connection established


...but it's only safe until someone accesses the script containing the class.  For added protection, consider using a package such as keyring or cryptography to secure your passwords.  Their usage is outside the scope of this post, however.


After creating a connection object you have a few options.  You can write queries and use Panda's `read_sql()` function to store the results in a DataFrame, as seen below.

In [4]:
# Execute a query and store results in a dataframe
query = """
SELECT TOP 5 *
FROM AP..Vendors
"""

vendors = pd.read_sql(query, conn)
vendors

,VendorID,VendorName,VendorAddress1,VendorAddress2,VendorCity,VendorState,VendorZipCode,VendorPhone,VendorContactLName,VendorContactFName,DefaultTermsID,DefaultAccountNo
0,1,US Postal Service,Attn: Supt. Window Services,PO Box 7005,Madison,WI,53707,(800) 555-1205,Alberto,Francesco,1,552
1,2,National Information Data Ctr,PO Box 96621,None,Washington,DC,20090,(301) 555-8950,Irvin,Ania,3,540
2,3,Register of Copyrights,Library Of Congress,None,Washington,DC,20559,None,Liana,Lukas,3,403
3,4,Jobtrak,1990 Westwood Blvd Ste 260,None,Los Angeles,CA,90025,(800) 555-8725,Quinn,Kenzie,3,572
4,5,Newbrige Book Clubs,3000 Cindel Drive,None,Washington,NJ,07882,(800) 555-9980,Marks,Michelle,4,394


Alternatively, if you're using a Jupyter notebook you can perform the query directly in the cell.  First, however, you need to make sure you have the ipython-sql package installed.  If you aren't sure, run `pip show ipython-sql` in your shell (Bash, PowerShell, Command Prompt, etc.) or run `!pip show ipython-sql` in a code cell in a Jupyter notebook.  Should you need to install it, all you need to do is run `pip install ipython-sql` in your shell or `!pip install ipython-sql` in a Jupyter notebook cell.

Once ipython-sql is installed you can run queries in code cells using a magic command.  You can choose between making a query in a single line or using an entire code cell.  First, however, you need to load the SQL extension in a separate code cell and use the connection string we wrote earlier to establish a connection.

In [5]:
%load_ext sql

%sql mssql+pyodbc://MATT-PC/AP?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server

### Code Cell Queries
Using code cell queries allows you to have queries that span multiple lines.  Using multiple lines when writing queries is usually preferable, since the added space allow you to organize your queries better and make them more readable.  If you've ever had to review old code, whether someone else's or your own, you know how much of a headache it can be if it isn't organized in a way that makes it clear what each block of code does.

With the preamble out of the way, let's get onto the good stuff.  All you need to do to convert a code cell to a multi-line SQL query is put `%%sql` on the first line.  After that, type out your query and run the cell.

In [6]:
%%sql

SELECT TOP 5 VendorName, InvoiceTotal, InvoiceDate, (InvoiceTotal - PaymentTotal - CreditTotal) AS BalanceDue
FROM Invoices JOIN Vendors
    ON Invoices.VendorID = Vendors.VendorID
ORDER BY (InvoiceTotal - PaymentTotal - CreditTotal) DESC;

 * mssql+pyodbc://MATT-PC/AP?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


VendorName,InvoiceTotal,InvoiceDate,BalanceDue
Malloy Lithographing Inc,20551.1800,2020-01-23,19351.1800
Malloy Lithographing Inc,10976.0600,2020-01-31,10976.0600
Ingram,579.4200,2020-01-21,579.4200
Ford Motor Credit Company,503.2000,2020-01-24,503.2000
Blue Cross,224.0000,2020-02-01,224.0000


This provides a viable alternative to using the IDEs that accompany each flavor of SQL.  Using Jupyter (or an IDE that supports .ipynb files like VS Code), you only have to learn the differences in syntax and commands between each flavor and not worry about learning how to use a new IDE.

Futhermore, while this is a useful interface for performing queries, this can also be used as a testing bed for writing queries to use in creating DataFrames with `pd.read_sql()` so you can further manipulate the data.

In [15]:
query = """
SELECT TOP 5 VendorName, InvoiceTotal, InvoiceDate, (InvoiceTotal - PaymentTotal - CreditTotal) AS BalanceDue
FROM Invoices JOIN Vendors
    ON Invoices.VendorID = Vendors.VendorID
ORDER BY (InvoiceTotal - PaymentTotal - CreditTotal) DESC;
"""

pd.read_sql(query, conn)

,VendorName,InvoiceTotal,InvoiceDate,BalanceDue
0,Malloy Lithographing Inc,20551.18,2012-03-23,19351.18
1,Malloy Lithographing Inc,10976.06,2012-03-31,10976.06
2,Ingram,579.42,2012-03-21,579.42
3,Ford Motor Credit Company,503.20,2012-03-24,503.20
4,Blue Cross,224.00,2012-04-01,224.00


Don't forget that DataFrames can be saved as CSVs and a variety of other useful formats.  This allows you to continue working with that DataFrame, use it in another program such as Tableau, or share it with others.

### Single Cell Queries
Beyond using a code cell for 

# Still to add!
* Single line queries
    * Storing results in variable
* Links to other useful articles


# Next SQL + Python article
* SQL data types
* Creating databases and tables
* Inserting data into tables
* DROP IF EXISTS


# Intermediate SQL Article
* Aliases
* GROUP BY & HAVING
* Joins
* CTEs
* Partition
* Roll-Up, etc.
* Subqueries
* Cursors
* Views